In [1]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0430 16:29:32.711215 140222199604992 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [4]:
!mkdir bert_output

mkdir: cannot create directory ‘bert_output’: File exists


In [5]:
OUTPUT_DIR = "bert_output"

In [6]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_output'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_output *****


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [7]:
from tensorflow import keras
import os
import re

In [8]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Words often confused" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [9]:
modelname = "OftenConfused_ToFifteen" # @param {type:"string"}

In [10]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [11]:
len(trn)

13363

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [12]:
train = trn
test = tst

In [13]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [14]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [15]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [16]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
5        <bert.run_classifier.InputExample object at 0x...
6        <bert.run_classifier.InputExample object at 0x...
7        <bert.run_classifier.InputExample object at 0x...
8        <bert.run_classifier.InputExample object at 0x...
9        <bert.run_classifier.InputExample object at 0x...
10       <bert.run_classifier.InputExample object at 0x...
11       <bert.run_classifier.InputExample object at 0x...
12       <bert.run_classifier.InputExample object at 0x...
13       <bert.run_classifier.InputExample object at 0x...
14       <bert.run_classifier.InputExample object at 0x...
15       <bert.run_classifier.InputExample object at 0x...
16       <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [17]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 16:30:00.612164 140222199604992 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [18]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [19]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 13363


I0430 16:30:21.499293 140222199604992 tf_logging.py:115] Writing example 0 of 13363


INFO:tensorflow:*** Example ***


I0430 16:30:21.503939 140222199604992 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0430 16:30:35.255942 140222199604992 tf_logging.py:115] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0430 16:30:35.257216 140222199604992 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0430 16:30:35.258447 140222199604992 tf_logging.py:115] label: 0 (id = 0)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [20]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [21]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [22]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [23]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

2504


In [24]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [25]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_task_id': 0, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_save_checkpoints_secs': None, '_num_worker_replicas': 1, '_device_fn': None, '_master': '', '_task_type': 'worker', '_eval_distribute': None, '_experimental_distribute': None, '_train_distribute': None, '_save_summary_steps': 100, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_is_chief': True, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'bert_output', '_tf_random_seed': None, '_save_checkpoints_steps': 500, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f876ea324a8>, '_log_step_count_steps': 100}


I0430 16:30:38.981182 140222199604992 tf_logging.py:115] Using config: {'_global_id_in_cluster': 0, '_task_id': 0, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_save_checkpoints_secs': None, '_num_worker_replicas': 1, '_device_fn': None, '_master': '', '_task_type': 'worker', '_eval_distribute': None, '_experimental_distribute': None, '_train_distribute': None, '_save_summary_steps': 100, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_is_chief': True, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'bert_output', '_tf_random_seed': None, '_save_checkpoints_steps': 500, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f876ea324a8>, '_log_step_count_steps': 100}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [26]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [27]:
print('Beginning Training!')
current_time = datetime.now()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/gpu:0'):
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0430 16:30:45.243468 140222199604992 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 16:30:48.640784 140222199604992 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 2504 into bert_output/model.ckpt.


I0430 16:47:56.194984 140222199604992 tf_logging.py:115] Saving checkpoints for 2504 into bert_output/model.ckpt.


INFO:tensorflow:Loss for final step: 0.0032409686.


I0430 16:47:59.938127 140222199604992 tf_logging.py:115] Loss for final step: 0.0032409686.


Training took time  0:17:20.933721


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [28]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [29]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0430 16:51:37.488837 140222199604992 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 16:51:41.824554 140222199604992 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 2504: auc = 0.65070856, eval_accuracy = 0.9395391, f1_score = 0.39880946, false_negatives = 142.0, false_positives = 60.0, global_step = 2504, loss = 0.40514794, precision = 0.52755904, recall = 0.32057416, true_negatives = 3072.0, true_positives = 67.0


I0430 16:52:09.546745 140222199604992 tf_logging.py:115] Saving dict for global step 2504: auc = 0.65070856, eval_accuracy = 0.9395391, f1_score = 0.39880946, false_negatives = 142.0, false_positives = 60.0, global_step = 2504, loss = 0.40514794, precision = 0.52755904, recall = 0.32057416, true_negatives = 3072.0, true_positives = 67.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2504: bert_output/model.ckpt-2504


I0430 16:52:12.902090 140222199604992 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 2504: bert_output/model.ckpt-2504


{'auc': 0.65070856,
 'eval_accuracy': 0.9395391,
 'f1_score': 0.39880946,
 'false_negatives': 142.0,
 'false_positives': 60.0,
 'global_step': 2504,
 'loss': 0.40514794,
 'precision': 0.52755904,
 'recall': 0.32057416,
 'true_negatives': 3072.0,
 'true_positives': 67.0}

#Extracting the trained model


Now let's save our model:

In [0]:
# Export the model
def serving_input_fn():
  with tf.variable_scope("foo"):
    feature_spec = {
        "input_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "input_mask": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "segment_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "label_ids": tf.FixedLenFeature([], tf.int64),
      }
    serialized_tf_example = tf.placeholder(dtype=tf.string,
                                           shape=[None],
                                           name='input_example_tensor')
    receiver_tensors = {'examples': serialized_tf_example}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

EXPORT_DIR = './Exported models/'+modelname
estimator._export_to_tpu = False  # this is important
path = estimator.export_savedmodel(EXPORT_DIR, serving_input_fn)
print(path)

Check if we can load it correctly:

In [0]:
subdirs = [x for x in Path(EXPORT_DIR).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

In [0]:
from tensorflow.contrib import predictor

predict_fn = predictor.from_saved_model(latest)

In [30]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [31]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.red {\n\tbackground: #ffdddd;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="red">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [33]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/old IELTS/IELTS2015/EEm_10_2.txt": "The question of male and female studing is very interesting now. On the one hand, all people can get the high education, on the other hand, there are a lot of men on the phisical disciplines and a lot of women on the humanitarium disciplins. Is it a reason to make the same nomber of men and women on the faculties in the universities?\n\nIf we look on the one side of this problem, we will say actually \u00abYes!\u00bb. All people must have the same rules. But other\u2019s belief is that not all girls can studie economic disciplins, math or phisics. One the one hand this is a problem, but is it really?\n\nSo, let\u2019s have a look. For example, in the end of June in owe(?) country all youngh people, who finised education at school, give there documents to the universities. Almost girls choose literature, social sciences and so on and boys choose phisical faculties.\n\nIs it a problem of the university, that there a lot of male students in this subject and a lot of female student in the another.\n\nI am not sure that this is a problem of the university. On my faculty on cultural studies in the Higher School of economics there are a very little number of male students. But if the facultie take the same number of students both genders there would be only 6 men on the first course last year.\n\nBut if we believe scientists, all people are the same, and there are no problem of gender in our life, but in real life we see the other picture.\n\nAs for me there are no problem that there are less boys on my facultie. If I want to communicate with men, I must do it after my studie and the gender problem must\u2019n be interesting to me when I learn new words or read a text about philosophy of Kant. When you are in the university all students both genders must have the same gender. And it isn\u2019t a question of libertie of female rates, this is a problem of level of education.\n", "./data/exam/exam2014/EPa_83_2.txt": "Nowadays our society is concerned by the issue of public health. While some citizens claim that it is necessary to provide more sports facilities, others believe that it is not useful and that we need to solve this problem in an another way.\nAccording to the first point of view we need to expand the variety of sports for the society. It can be achieved by building new sports school or by reducing the price of attending section. The government also can contribute money in new equipment in schools or in the streets. People believe, that in this case sport will be available for all who want to improve their health. \nHowever, great public heath can be achieves by taking another measures. Firstly, the government can built new hospitals and offer free medical observation, for example, once a year. Secondly, the healthy way of life should be promoted on TV or by the celebrities. It can motivate young people too keep a diet or to go in for sports. The one another option is to around population for their results in improving their health. It can be some competitions with significant prizes. As a rule, people are motivated by desire to win something or to become the first among others. And, of course, we need to prevent air, and water pollution what influence our health a lot.\nIn conclusion I would like to say that it is not effective to offer some options if people do not want to participate in it. We need to wake their minds and to convince that our health is necessary and that we need to take care of it by ourselves.\n", "./data/exam/exam2014/DAr_38_1.txt": "In the paper there are four pie-charts. First two pie charts gives us information about the ages of the populations of Yemen in 2000 and projections for 2050. Second two the same information of populations in Italy. The first chart of Yemen shows us that approximetly half are younger people with percentage \u2013 50,1%. 46,3% are middle-aged (15-59 years). And only 3,6% are elder residents. But in projection for 2050 in Yemen of population the number of workable people will increase and will be 57,3%. The number of elder people also will raise \u2013 5,7%. But the number of children will decrease \u2013 37%. The situation in Italy in 2000 is different. The percentage of middle-aged people is big 61,6%. The amount of people in 14 years old and younger is 24,1%. And the number of elder residents is 14,3%. In 2050 the percentage of people, who in range of 15-59 years old will decrease and take the number of 46,2%. About 24,3% of the population will be young people. And pensioners takes only 11,5% of the population of Italy in 2050.\n", "./data/exam/exam2017_4/DPe_69_2.txt": "There is no doubt that copying and sharing films or music on the Internet is illegal. There are people who use illegal sites and methods. Others would like to punish such people and think it is a theft.\nTo my mind, the Internet is a huge unreal location for crime. Downlowding music, films and other types of media is illegal nowadays. What is more, we should not forget that special part of our society lose huge amounts of money because of pirate copies. These people do a really hard work producing films or music. This business is their work and they get money for it. Why does it allow to get their work and money bock? I consider, we have to punish people who download some files for free and using illegal methods. We should appreciate hard work of media-sphere.\nSome people wouldn\u2019t like to support my position because they use illegal methods by themselves. These people think that Internet is a free zone for everybody. They don\u2019t understand why they cannot use some sites with music or films. It is normal situation and we just have to explain them about some author\u2019s rights and pirate copies. We should not forget about punishments. If you copy or share music or films on the Internet, you have to pay fee or sometimes it can be more strict punishment.\nTo sum up, I would like to say that Internet is a free zone but with boerders. If you don\u2019t know these boarders and allow yourself to do what you want, you should be punished.\n", "./data/exam/exam2016/best_works/ZEv_10_2.txt": "Nowadays, the problem of air pollution and global warming takes first place in the list of humanity problems. Some people believe that air carries are responsible for air pollution more than car owners and factories and governments should take measures to reduce their harm to the atmosphere. However, I strongly disagree with this opinion.\n\nFirst of all, there is a huge demand in air travels. There are a lot of aims air carries can help to reach from private needs like desire of holidays or want to visit relatives to delievery of goods from another part of the world. Obviously, it is impossible to decrease the number of air travels.\n\nMoreover, greater contribution to air pollution and global warming lies in car owners and factories hands. Every day people themselves destroy atmosphere using private transport instead of public. It should be noticed that in modern world a lot of families have two and more cars for each family member. Therefore, there is a huge air polluting effect produced by ouselves. What is more, there is no need to discuss about responsibility of factories for damaging atmosphere.\n\nIn addition, there are not so many rich people in the World who can afford using their own flights. Thus, I cannot agree that a lot of air travel is unnecessary as private flights are only one of some hundreds of public ones.\n\nIn conclusion, there are some other spheres to control by governments but air travels. Everyday human's actions can make a greater input in destroying atmosphere and causing global warming. Therefore, governments should focus on creating laws to control people's usage of private transport and factories' pullings in rivers and air instead of reducing the amount of air travels used.\n", "./data/exam/exam2017_5_1/EGe_16_2.txt": "The number of teenagers who take part in crimes is getting bigger every year. In this essay I am going to write about the most powerful causes and try to make some suggestions how to deal with this problem.\nFirst of all, the biggest problem is that children have bad influence from TV shows, computer games and social networks. There are a great number of online and computer games, where you have to grap banks and kill people. Teenagers think that it is a good way to make money with crimes. Moreover, children kill people in computer games and it makes them agressive. About ten years ago, children did not have this opportunity to use computer every day, that is why they spent time with friends outside and play \"children games\". As a result, these games make them addicted and they want to make crime in a real life.\nSecondly, children have a lot of free time and they start to use internet very early. There are many online websites which force young people make crime. These groups make teenagers agressive and children think that it is normal to do crime for money or for your own reasons.\nIt is really difficult problems to be solved. Firstly, children should make sport, it will help them full there free time. Moreover, when you make sport you can find good friends who have the same interests as you are, communicate with them and spend your free time together on different sport competitions.\nSecondly, it is really important not to give computer or any gadgets with internet to young children. Parents and teachers should limit children's using od gadgets, because it is bad for their mind.\nTo sum up, the problem of teenagers' crime is important nowdays and adults should explaine young people that it is horrible to make crime.", "./data/exam/exam2014/MGr_6_2.txt": "We live in the modern, high-technologycal world with rapid and free access to different kind of information. However, it is quite difficult to define what happiness it.\nFrom my point of view, it goes without saying, that tastes differ. According to this, there is not only one answer to the question revealed in the introduction.\nNevertheless there are some stereotypes about the success, but has nothing against with the happiness. Somebody consider that the main goal in their life is creating the family, others think it is a good job. As for me, I suppose, happiness is not achievement, it is a feeling of freedom, love or looking for you child. Hapiness is something light, something that you can\u2019t describe as new flat, for example.\nThe main features of achieving these emotions are smile and kind, in my opinion. We should take it more seriosly, because happiness ensures the success. As for factors, which have influence on our feelings, I\u2019d like to pay attention to family, weather and health. These are the basis of hapiness.If you have lovely people, see sum and you are healthy, there is no problem which you can\u2019t reduce.\nTo sum up, I\u2019d like to reccommened not forget that majority of us is happy. \u201cYou don\u2019t lose your smile, you forget to use it\u201d. It is very good principle to achieve happiness.\n", "./data/exam/exam2017_4/DPe_59_2.txt": "Musical and film industry lose a lots of money in a modern times because of illegal copies and \u201cpirates\u201d attacks. The problem is that people nowadays prefer download some films and music albums from internent. As a result, musicians and film producers lose their money, because none of people want to buy their works.\nAs for me, I agree with this point of view. And I also will try to prove it. The first thing is, that musicians and film producers waste their time and nerves while makeing something new. A lot of my friends are musicians and they say that it is realy hard to record something good without having enough maney. Imagine how wasteful the film is: operators work, paying to actors etc. Pirate copies make musicians and film producers less motivated, because in the end they get almost no money for their work. The second thing is, that every musician need money. It is well known fact that domestic recordings are not so good. Recording in a studio costs a lot of money; the same thing with mastering a composition or song. Musicians hope that they will sell their recording for example in i-tunes and earn some money to make new music and new albums. This second fact shows that music industry is too wasteful like cinema and pirate copies destroy the enthusiasm of musician. The last fact about such topic is that concerts don\u2019t give enough money to cover all wastings. We can say that cinema industry has the same problems too.\nBut some people say that they don\u2019t need money to produce something new. For example in underground culture money don\u2019t play an important role. The concept of underground may seem strange. Films and music contains strong language and forbidden topics. It is clear that people won\u2019t pay any money for that. Pirate copies are not forbidden in underground; reverse, this culture adore this. Illegal copies make free advertisment for some musicians.\nTo sum up, I would like to say that illegal copies influence really bad on film producers and some musicians because as we can know from above these two industries needs a lot of money. But for others, pirate copies seems so good. It\u2019s their point of view. As for me, (as I said before) illegal copies will be always bad thing in internet.\n", "./data/Exam_practice/AV_1_year/Test_essays/student68_final.txt": "Nowadays there are a lot of disputes about company\u2019s success and ways of getting it. So, is it depend of the happiness of its employees?\nI truly believe that it does. Firstly, people how are happy will be more interested in their work, than those how are sad would prefer to think about their troubles. Secondly, happy people will make more efforts to do their work, while disappointed people will do it as far as they were ordered.  At last, people how are in troubles would like to be noticed, so they will be talking with others and it is very unprofessional and sometimes even dangerous for all.\nHowever, it has to be said that sometimes happy people could be more useful than others. For example, they can be so interested in themselves that they do not see anything and anybody around them.\nTo conclude, I can say that both happy and unhappy people could have success in their work, but probably, those who are happy have a little bit more chances. \n", "./data/exam/exam2017/EGe_15_1.txt": "We have two graphics, which show us the population in the USA and birth and death rate from 1750 to 2000.\nAs we can see in the first graph, the growth of the USA population gradually rise. Besides, during the last part of the 20th century, there are was more intensive growth. That is probably means that there were more babies born or people immigrated to the USA.\nThe second graph shows us that during all this period birth and death rates were not stable.There was a pick of birth and death activity during the second part of 18th century, after that the graphs gradualy fall. For the last 50 years of the 20th century the number of deaths is stable, while the births are growing. \tIf we compare this two graphs we can make a conclusion that the sharp increase of population is mostly caused by newcomers, who moved to the USA after The Second World War.", "./data/Exam_practice/OV_2_year/essays_fr/st_43_1.txt": "The given bar chart shows the estimated world literacy rates for the 6 areas by gender and region for the year 2020.\n\nAs we can see, the most educated and smart people are believed to live in the South Asia, and there are more intelligent women there than men. Approximately the same situation can be found in the Arab States and in the Sub-Saharan Africa.\n\nIn the developed countries the proportion of literacy among male and female is almost similar, but speaking about the Latin America and East Asia we can see that the number of smart women in these areas dominates over the number of educated men.\n\nAll in all, the female literacy is much higher almost in all areas except Latin America, and the developed countries in which we can see roughly the similar proportions. It is also important to note that according to the chart the most illiterate people are people from the developed countries.\n", "./data/exam/exam2017/OBy_61_1.txt": "We can see two nice schemes on the list with clear round form. People mostly choose a circle then we ask them about \u2018sexy shape\u2019, and it\u2019s a reason why we looking at them and getting esthetic happiness. Do we look on the information on graphs? Oh no, we just like these geometric patterns in forms and white space around circle. Its like a new planet in the nowhere; like ours wishes on the black strong matery behind the human death. Goodsm that transporting towards clear air around our 3 meters spaces the ground.\n\nThere\u2019s a metals for example. It\u2019s our holy money with boring parallel lines pulse our heart then we see cash. By rail we lost 35% money, its bigger then by road because RZD needs more capital for new any politics dacha for example. In the road move chemicals then in rail because drugs an auto in auto is American classic. But just see on food products. We less ear in rail, because \u2018doshirak\u2019 isn\u2019t food.\n\nIf we compare charts, we see that important is one thing and it\u2019s a colorfull patterns. No matter, what human prefer \u2013 rail or road, metals of food. Untill we have a big white space around it and view of any person, these deathers for foolish, who cant the the whole system.\n", "./data/old IELTS/IELTS2016/JSl_3_1.txt": "In the chart we can see how number of children without acess to primary education by region and gender. For all years we see, that girls without access a lots then boys, but this rule have one exeption \u2013 South Asia 2012. Its wonderfully, but from 2000 to 2012 near by 40 million children has got acess to school education. Next we see some some details. In South Asia 2000 girls, who has no acess some less, than Africa 2000. But in South Asia 2012 girls, who has no acess more then 12 million less, then South Asia 2012. This mean, that South Asia really solve this problem.\nIn finnaly 1 give one interesting fact: all over the world 2012 children , who has no acess, more less, than one Africa . This work in 2000 year too. And, tfust, we have positive tendention, this mean, that world community solve this case.\n", "./data/exam/exam2014/DAr_11_1.txt": "The pie charts describe the different age groups of the population of two countries such as Yemen and Italy in 2000 and give predictions for 2050. The structure of age groups of Italian population differs from the Yemen\u2019s one in 2000. For example, there are only 14,3% of people, who are under 14 years old in Italy, while young people in Yemen take approximately 50%. But the percentage of sixty years old and older is higher in Italy than in Yemen. Also the parts of middle age groups are nearly the same in both countries. To compare the predictive data, there are few changes. In Yemen the part of middle age group will become bigger in 2050, than in 2000. But in Italy there is an alternative picture. Moreover the part of young people will be reduce in both countries. In addiction, the part of eldery people will be rised in Yemen for 2,1% and in Italy for about 20%.\n", "./data/exam/exam2014/EPa_4_2.txt": "It is certainly true, that growth of quantity of sports facilities can improve public heath, but it is not the best way to do it.\nSport centers and sport shops can reduce prices for sport equipment or for club cards to do different sports. There is no evidence that someone is not interested in some sports, but it can be difficult to buy equipment. For instance, hockey defence for goalkeeper has the enourmous price. Secondly, government can provide different policies which will make investors more interested in building sport areas or sport center for everyone. Also government can make it free or with some preveuligues for special groups of people.\nOn the other hand, increasing the number of sport facilities will not help to improve people`s health in significant way. Also it depends on people. People who interested in sports but do not want to do it will not feel the increasing number of sport facilities. There are other better ways to improve public health. For instance, governments can also provide policies which will have different restrictions for alcohol or smoking. As we know, modern world has such examples and you can find it in history. Public health depends on mentality and worldsight of people. Also sport fashion has the influense. For example, during the Olympic games, young people become more interested in hockey and they wanted to play.\nIn conclusion, i want to say that health improvement depends on differents things and everybody and every country should choose the right and optimal way to do it with help of facilities or something else.\n"}
outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 128


I0430 16:56:01.683614 140222199604992 tf_logging.py:115] Writing example 0 of 128


INFO:tensorflow:*** Example ***


I0430 16:56:01.686070 140222199604992 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0430 16:57:49.980815 140222199604992 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0430 16:57:50.068684 140222199604992 tf_logging.py:115] Done running local_init_op.




Annotation took time  0:00:07.294386


In [34]:
os.listdir(".")

['.ipython',
 '.cache',
 '.ipynb_checkpoints',
 'delete.json',
 '.jupyter',
 '.bashrc-anaconda3.bak',
 '.config',
 'REALEC_AutoAnnotator_BERT_pipeline Often_confused.ipynb',
 'wsites_OftenConfused_ToFifteen.json',
 'Often_confused_test.json',
 'Often_confused.json',
 'anaconda3',
 'Anaconda3-4.0.0-Linux-x86_64.sh',
 'download_google_drive',
 '.nano',
 '.bash_history',
 'gdrive-linux-x64',
 '.keras',
 'delete_test.json',
 '.python_history',
 'nltk_data',
 'wsites_delete_ToFifteen.json',
 'Datasets',
 '.local',
 'wsites_lex_item_choice_AugmentedRatio.json',
 'jupyter-drive',
 'cuda-repo-ubuntu1604_9.0.176-1_amd64.deb',
 '.continuum',
 '.conda',
 '.bashrc',
 '.nv',
 'bert_output',
 'cuda-repo-ubuntu1604_8.0.61-1_amd64.deb',
 '.gnupg',
 'REALEC_AutoAnnotator_Generic_BERT_pipeline_Instance_2-Copy1.ipynb',
 '.gdrive',
 'lex_item_choice_test.json',
 '.profile',
 '.wget-hsts',
 'REALEC_AutoAnnotator_Generic_BERT_pipeline_Instance_2.ipynb',
 '.bash_logout',
 '.ssh',
 'lex_item_choice.json']

In [35]:
outie[3]

'<html>\n<head>\n<title>./data/exam/exam2014/MGr_6_2.txt</title>\n<meta charset="utf-8">\n<style type="text/css">\n.red {\n\tbackground: #ffdddd;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>We <div class="red">live</div> in the modern, high-technologycal world with rapid and free access to different kind of information. However, it is <div class="red">quite</div> difficult to define <div class="red">what</div> happiness it. From my point of view, it goes without saying, that tastes differ. According to this, there is not only one answer to the question <div class="red">revealed</div> in the introduction. Nevertheless there are some stereotypes about the success, but has nothing against with the happiness. Somebody consider that the main <div class="red">goal</div> in their life is creating the family, others think it is a good job. As for me, I suppose, happiness is not achievement, it is a feeling of freedom, love or looking for you child. Hapiness is something <div class="

In [36]:
!./gdrive-linux-x64 upload wsites_OftenConfused_ToFifteen.json

Uploading wsites_OftenConfused_ToFifteen.json
Uploaded 1qOAcAs30Ii1nPHYmD3sN_5U97FqQKYDH at 16.7 KB/s, total 26.2 KB


In [37]:
!ls -lAh ./bert_output

total 6.5G
-rw-rw-r-- 1 ivantorubarov ivantorubarov  271 Apr 30 16:47 checkpoint
drwxr-xr-x 2 ivantorubarov ivantorubarov 4.0K Apr 30 16:52 eval
-rw-rw-r-- 1 ivantorubarov ivantorubarov  49M Apr 30 16:19 events.out.tfevents.1556640968.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov 100M Apr 30 16:47 events.out.tfevents.1556641872.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov  89M Apr 30 16:31 graph.pbtxt
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 16:38 model.ckpt-1000.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K Apr 30 16:38 model.ckpt-1000.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  26M Apr 30 16:38 model.ckpt-1000.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 16:41 model.ckpt-1500.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K Apr 30 16:41 model.ckpt-1500.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  26M Apr 30 16:41 model.ckpt-1500.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 16:44 model.ckpt-

In [38]:
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2504.data-00000-of-00001
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2504.index
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2504.meta

Uploading ./bert_output/model.ckpt-2504.data-00000-of-00001
Uploaded 14XnUaNHerYMyX6I-aq_Ki5ioVlQO3Q8K at 34.9 MB/s, total 1.3 GB
Uploading ./bert_output/model.ckpt-2504.index
Uploaded 1fQJl_iJiFmGuiOV8jxwnnHsTWluH6-yW at 23.5 KB/s, total 23.5 KB
Uploading ./bert_output/model.ckpt-2504.meta
Uploaded 1bqvGqLeV4MW36A3J2pdJo-ykovsd4C48 at 16.0 MB/s, total 26.5 MB


In [55]:
!rm -rf ./bert_output

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd


# Preparing existing model for REALEC production